In [6]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import KFold

# Generate a sample bearing dataset
data = {
    'static_capacity_kgf': [100, 200, 150, 300, 250, 350, 400, 450, 500, 550],
    'dynamic_capacity_kgf': [50, 100, 75, 150, 125, 175, 200, 225, 250, 275],
    'max_speed_rpm': [1000, 1500, 1200, 2000, 1700, 2200, 2500, 2700, 3000, 3500],
    'type_of_bearing': ['Ball', 'Roller', 'Needle', 'Thrust', 'Tapered', 'Ball', 'Roller', 'Needle', 'Thrust', 'Tapered'],
    'bearing_number': ['6205', '22205', 'NA4905', '51105', '30205', '6206', '22206', 'NA4906', '51106', '30206'],
    'dimensions': ['25x52x15[d,D,r]', '25x52x18', '25x42x17', '25x47x18', '25x52x16', '30x62x16', '30x62x20', '30x47x17', '30x55x18', '30x62x17']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Encode categorical variables
label_encoders = {}
for column in ['type_of_bearing', 'bearing_number', 'dimensions']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Split the data into features and target variable
X = df[['static_capacity_kgf', 'dynamic_capacity_kgf', 'max_speed_rpm']]
y = df['type_of_bearing']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 3000],
    'tol': [1e-3, 1e-4, 1e-5]
}

# Create a SGDClassifier
sgd = SGDClassifier(loss='hinge')

# Perform GridSearchCV with fewer splits
kf = KFold(n_splits=2)
grid_search = GridSearchCV(sgd, param_grid, cv=kf, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters from GridSearchCV
print(f'Best parameters: {grid_search.best_params_}')

# Train the model with best parameters
best_sgd = grid_search.best_estimator_
best_sgd.fit(X_train, y_train)

# Function to predict the suitable bearing
def suggest_bearing(static_capacity, dynamic_capacity, max_speed):
    # Create the input array
    input_array = [[static_capacity, dynamic_capacity, max_speed]]

    # Standardize the input array
    input_array = scaler.transform(input_array)

    # Predict the bearing
    predicted_type_of_bearing = best_sgd.predict(input_array)
    type_of_bearing_label = label_encoders['type_of_bearing'].inverse_transform(predicted_type_of_bearing)[0]

    bearing_data = df[df['type_of_bearing'] == predicted_type_of_bearing[0]].iloc[0]

    bearing_number = label_encoders['bearing_number'].inverse_transform([bearing_data['bearing_number']])[0]
    dimensions = label_encoders['dimensions'].inverse_transform([bearing_data['dimensions']])[0]

    return type_of_bearing_label, bearing_number, dimensions

# GUI Application using tkinter
class BearingSuggestionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Bearing Suggestion System")

        self.create_widgets()

    def create_widgets(self):
        ttk.Label(self.root, text="Static Capacity (kgf):").grid(row=0, column=0, padx=10, pady=5)
        self.static_capacity = ttk.Entry(self.root)
        self.static_capacity.grid(row=0, column=1, padx=10, pady=5)

        ttk.Label(self.root, text="Dynamic Capacity (kgf):").grid(row=1, column=0, padx=10, pady=5)
        self.dynamic_capacity = ttk.Entry(self.root)
        self.dynamic_capacity.grid(row=1, column=1, padx=10, pady=5)

        ttk.Label(self.root, text="Maximum Speed (rpm):").grid(row=2, column=0, padx=10, pady=5)
        self.max_speed = ttk.Entry(self.root)
        self.max_speed.grid(row=2, column=1, padx=10, pady=5)

        self.submit_button = ttk.Button(self.root, text="Suggest Bearing", command=self.suggest_bearing)
        self.submit_button.grid(row=3, column=0, columnspan=2, pady=10)

    def suggest_bearing(self):
        try:
            static_capacity = float(self.static_capacity.get())
            dynamic_capacity = float(self.dynamic_capacity.get())
            max_speed = float(self.max_speed.get())

            type_of_bearing, bearing_number, dimensions = suggest_bearing(static_capacity, dynamic_capacity, max_speed)
            messagebox.showinfo("Suggested Bearing", f"Type: {type_of_bearing}\nBearing Number: {bearing_number}\nDimensions: {dimensions}")
        except ValueError:
            messagebox.showerror("Invalid input", "Please enter valid numeric values for capacities and speed.")

if __name__ == "__main__":
    root = tk.Tk()
    app = BearingSuggestionApp(root)
    root.mainloop()


Best parameters: {'alpha': 0.0001, 'max_iter': 1000, 'tol': 0.001}


/Users/rujaiananth/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
